# Thrive assignment notebook

In [ ]:
import pandas as pd
import sqlite3

In [3]:
connection = sqlite3.connect("thrive_test_db.db")

In [4]:
# Create a cursor object
cursor = connection.cursor()

In [5]:
# Get all table objects
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the tables
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
conversation_start
conversation_parts
users
consolidated_messages


In [6]:
cursor.execute("SELECT * FROM users;")

# Fetch ALL results
rows = cursor.fetchall()
print("Rows in the 'fish' table:")
for row in rows:
    print(row)

Rows in the 'fish' table:
('Douglas Roman', 'douglas.roman@example.com', 0, 1)
('Carla Clark', 'carla.clark@example.com', 1, 1)
('Maria Reed', 'maria.reed@example.com', 2, 1)
('Ashley Nelson', 'ashley.nelson@example.com', 3, 1)
('Alexandra Richardson', 'alexandra.richardson@example.com', 4, 1)
('Cheryl Johnson', 'cheryl.johnson@example.com', 5, 1)
('Robert Flores', 'robert.flores@example.com', 6, 0)
('Anita Barron', 'anita.barron@example.com', 7, 1)
('Anthony Callahan', 'anthony.callahan@example.com', 8, 0)
('Cody Bass', 'cody.bass@example.com', 9, 1)
('Teresa Mitchell', 'teresa.mitchell@example.com', 10, 1)
('Benjamin Smith', 'benjamin.smith@example.com', 11, 1)
('Jeremiah Mills', 'jeremiah.mills@example.com', 12, 1)


In [7]:
users_table = "users"
conversations_start = "conversation_start"
conversations_part = "conversation_parts"

query = f"SELECT * FROM {users_table};"
users = pd.read_sql_query(query, connection)

query = f"SELECT * FROM {conversations_start};"
conversation_start = pd.read_sql_query(query, connection)

query = f"SELECT * FROM {conversations_part};"
conversations_part = pd.read_sql_query(query, connection)

In [8]:
users

,name,email,id,is_customer
0,Douglas Roman,douglas.roman@example.com,0,1
1,Carla Clark,carla.clark@example.com,1,1
2,Maria Reed,maria.reed@example.com,2,1
3,Ashley Nelson,ashley.nelson@example.com,3,1
4,Alexandra Richardson,alexandra.richardson@example.com,4,1
5,Cheryl Johnson,cheryl.johnson@example.com,5,1
6,Robert Flores,robert.flores@example.com,6,0
7,Anita Barron,anita.barron@example.com,7,1
8,Anthony Callahan,anthony.callahan@example.com,8,0
9,Cody Bass,cody.bass@example.com,9,1


In [9]:
conversation_start

,created_at,source_type,id,priority,name,conv_dataset_email,message
0,1679498647,conversation,10065,not_priority,"[Alexandra, Richardson]",alexandra.richardson@example.com,Memory support seek ok size. Politics generati...
1,1679513798,conversation,10106,not_priority,"[Anita, Barron]",anita.barron@example.com,Both necessary test available. Until continue ...
2,1679602052,conversation,10222,not_priority,"[Cheryl, Johnson]",cheryl.johnson@example.com,Turn important similar arrive. Marriage agree ...
3,1679688990,conversation,10308,not_priority,"[Cody, Bass]",cody.bass@example.com,Measure anything production cup explain well. ...
4,1679865951,conversation,10342,not_priority,"[Benjamin, Smith]",benjamin.smith@example.com,Rich already rise head then executive. They ha...
5,1725885557,conversation,100120,not_priority,"[Carla, Clark]",carla.clark@example.com,Officer perform instead yes. Whatever protect ...
6,1726167821,conversation,102013,not_priority,"[Ashley, Nelson]",ashley.nelson@example.com,Decision north describe majority explain guess...
7,1726447265,conversation,102837,not_priority,"[Teresa, Mitchell]",teresa.mitchell@example.com,Class occur put spend.\nEnergy your reveal do ...
8,1679875562,conversation,10350,not_priority,"[Maria, Reed]",maria.reed@example.com,Energy arrive collection heart particularly co...
9,1679941011,conversation,10404,not_priority,"[Anthony, Callahan]",anthony.callahan@example.com,Vote evidence nor rate power fund. Stock once ...


In [10]:
conversations_part

,created_at,conversation_id,part_type,id,conv_dataset_email,message
0,1679498651,10065,comment,20482598151,robert.flores@example.com,Memory support seek ok size. Politics generati...
1,1679498716,10065,assignment,20482622915,anthony.callahan@example.com,Both necessary test available. Until continue ...
2,1679498908,10065,comment,20482707054,alexandra.richardson@example.com,Turn important similar arrive. Marriage agree ...
3,1679499097,10065,comment,20482788524,anthony.callahan@example.com,Measure anything production cup explain well. ...
4,1679499376,10065,comment,20482911573,alexandra.richardson@example.com,Rich already rise head then executive. They ha...
...,...,...,...,...,...,...
80,1726494982,102837,open,21904986991,teresa.mitchell@example.com,Have candidate campaign area skill news no beh...
81,1726495021,102837,comment,21904986992,anthony.callahan@example.com,Play oil focus whole place yeah. Worry list bi...
82,1726495022,102837,close,21904986994,anthony.callahan@example.com,Bag certain between air week subject answer pa...
83,1726495252,102837,open,21904986998,teresa.mitchell@example.com,When boy reason enjoy region choose career. In...
